In [77]:
import pandas as pd
import numpy as np

I want to explore some data. Let's use data about Tourism and Economic Impact Dataset that is available here: https://www.kaggle.com/datasets/bushraqurban/tourism-and-economic-impact
I've downloaded it into data folder

In [344]:
df = pd.read_csv('data/world_tourism_economy_data.csv')
df.head(10)
len(df)

6650

I want to visualize some data. Let's plot country name along with gdp and tourism receipts. Next I want to plot how much % of gdp consists of tourism

But to do that I need to learn basic of matplotlib

In [31]:
tdf = df[df.country != 'World']
tdf[['country', 'gdp', 'tourism_receipts']].sort_values(by='gdp', ascending=False).head(30)

,country,gdp,tourism_receipts
6479,High income,6.827649e+13,NaN
6213,High income,6.496126e+13,NaN
6565,OECD members,6.474656e+13,NaN
5947,High income,6.273789e+13,NaN
6299,OECD members,6.069133e+13,NaN
6582,Post-demographic dividend,6.048525e+13,NaN
6033,OECD members,5.916960e+13,NaN
5415,High income,5.735101e+13,1.152147e+12
6316,Post-demographic dividend,5.706675e+13,NaN
5149,High income,5.679396e+13,1.147991e+12


There are many rows with country value which's not a country. I want to fix it using some file with all countries in the world.

I want to turn country list into convenient csv file, and then into pandas dataframe

In [394]:
with open('data/all_countries_in_the_world.txt') as src, open('data/all_countries_in_the_world.csv', 'w') as dest:
    for line in src:
        stripped_line = line.strip()
        if stripped_line:
            dest.write(stripped_line + ',')
countries = pd.read_csv('data/all_countries_in_the_world.csv')

In [34]:
countries

,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Vatican City (Holy See),Venezuela,Vietnam,Yemen,Zambia,Zimbabwe,Unnamed: 201


consider only those years where's data about tourism receipts

In [529]:
dtoplt = df[['country', 'gdp', 'tourism_receipts', 'year']] #data to plot
dtoplt = dtoplt[
    (dtoplt['country'].isin(countries)) &
    (dtoplt['tourism_receipts'].notna())
].sort_values(by='gdp', ascending=False)
len(dtoplt)

2829

Handle NaN gdp values. Let's put into it gdp from a year before

In [531]:
#find other rows and take that with year closest to year of row where we have NaN gdp
def fix_nan_gdp(country):
    nan_country = dtoplt.loc[(dtoplt['gdp'].isna()) & (dtoplt['country'] == country)]
    if len(nan_country):
        not_nan_countries = dtoplt.loc[(dtoplt['country'] == country) & ~(dtoplt['gdp'].isna())]

        years = not_nan_countries['year'].to_numpy()
        replacing_year = years[0]
        for nan_year in nan_country['year'].to_numpy():
            for year in years:
                if np.abs(year - nan_year) < np.abs(replacing_year - nan_year):
                    replacing_year = year
        #update NaN gdp
        gdp = not_nan_countries.loc[not_nan_countries['year'] == replacing_year]['gdp'].head(1).item()
        dtoplt.loc[(dtoplt.country == country) & (dtoplt.gdp.isna()), 'gdp'] = gdp
fix_nan_gdp('South Sudan')
fix_nan_gdp('Eritrea')



In [532]:
dtoplt[dtoplt['gdp'].isna()]

,country,gdp,tourism_receipts,year
